In [1]:
%load_ext autoreload



%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os


In [2]:
import os
GPU = '0'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]= GPU

In [3]:
import keras

Using TensorFlow backend.
/home/samer/miniconda3/envs/mod_framework/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/samer/miniconda3/envs/mod_framework/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/samer/miniconda3/envs/mod_framework/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [4]:
from frm_dataset_loader import load_dataset
from conf_dataset_1 import *


['ook', 'ask4', 'ask8', 'bpsk', 'qpsk', 'psk8', 'psk16', 'psk32', 'apsk16', 'apsk32', 'apsk64', 'qam16', 'qam32', 'qam64', 'gmsk', 'cpfsk']


In [5]:
fname =  'datasets/valid_1.dat'

(comb_valid,carrier_valid,clean_valid,fading_valid,_,
         freq_valid,timing_offNum_valid,timing_step_valid,
         _,mod_valid,snr_valid) = load_dataset(fname,max_sps,len(mod_list))

In [6]:
fname = 'datasets/test_1.dat'

(comb_test,carrier_test,clean_test,fading_test,raw_test,
         freq_test,timing_offNum_test,timing_step_test,
         coeff_test,mod_test,snr_test) = load_dataset(fname,max_sps,len(mod_list))

In [7]:
from frm_train_generator import train_generator_mod as train_generator

In [8]:
from frm_nn_baseline import *

In [9]:
FNAME = '004'

In [10]:
indx_valid = slice(None)

batch_size = 128

samples_per_epoch = 800000

gen = train_generator(samples_per_epoch,batch_size,pkt_size,max_sps,mod_list,
                   sps_rng,pulse_ebw_list,timing_offset_rng,fading_spread_rng,freq_err_rng,phase_err_rng,snr_rng)
gen.__getitem__(0)
patience = 10
n_epochs = 400
nworkers = 10

/home/samer/Documents/mod_framework_journal/frm_modulations_fast.py:143: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(complex64, 1d, A), array(complex64, 1d, C))
  y[k] = np.dot(pulse_vec,symbs_vec)


In [ ]:
gru = create_model_gru(len(mod_list),pkt_size)

In [ ]:
filepath = f'models/{FNAME}_gru.h5'
c=[ keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True),
  keras.callbacks.EarlyStopping(monitor='val_loss',  patience=patience)]


gen.first_run = True

gru_history = gru.fit_generator(gen,
                            epochs = n_epochs,callbacks=c,
                            workers=nworkers, use_multiprocessing=True,
                            validation_data = 
                            (comb_valid[indx_valid], mod_valid[indx_valid]))

In [ ]:
import datetime

outputs = {}
outputs['dataset_params'] = [pkt_size,max_sps,mod_list,sps_rng,pulse_ebw_list,timing_offset_rng,fading_spread_rng,freq_err_rng,phase_err_rng,snr_rng]
outputs['date'] =  f'{datetime.datetime.now():%Y-%m-%d %H:%M:%S%z}'
outputs['history'] = [gru_history.history]
outputs['train_params'] = [patience,n_epochs]

In [ ]:
import pickle
with open(f'outputs/{FNAME}.pkl','wb') as f:
    pickle.dump(outputs,f)

In [12]:
import pickle
with open(f'outputs/{FNAME}.pkl','rb') as f:
    outputs = pickle.load(f)
[gru_history.history]=outputs['history']


ValueError: not enough values to unpack (expected 5, got 1)